In [1]:
import wikipedia
import pandas as pd
import spacy

In [2]:
df_en = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\train_ds\mushroom.en-train_nolabel.v1.jsonl', lines=True) #Cambiar para directorio local
df_en.head(5)

,lang,model_id,model_input,model_output_text,model_output_logits,model_output_tokens
0,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
1,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids have at least ...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
2,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
3,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
4,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."


In [3]:
df_es = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\train_ds\mushroom.es-train_nolabel.v1.jsonl', lines=True)
df_es.head(5)

,lang,model_input,model_id,model_output_tokens,model_output_logits,model_output_text
0,ES,¿Dónde está ubicado Cilleruelo de San Mamés?,meta-llama/Meta-Llama-3-8B-Instruct,"[C, iller, uel, o, Ġde, ĠSan, ĠMam, Ã©s, Ġes, ...","[25.4848022461, 31.8824501038, 37.6627693176, ...",Cilleruelo de San Mamés es un municipio situad...
1,ES,¿Cuándo fue creada la comuna suiza Gampel-Brat...,meta-llama/Meta-Llama-3-8B-Instruct,"[La, Ġcom, una, Ġsu, iza, Ġde, ĠG, amp, el, -,...","[20.9607429504, 21.4772224426, 31.493303299, 2...",La comuna suiza de Gampel-Bratsch fue creada e...
2,ES,¿Cuál es el único continente en el que habitan...,meta-llama/Meta-Llama-3-8B-Instruct,"[La, Ġrespuesta, Ġes, ĠAustralia, ., ĠLos, Ġco...","[20.0091133118, 20.0120868683, 23.2712993622, ...",La respuesta es Australia. Los corvinos negros...
3,ES,¿Qué significa la abreviatura IOMMU?,meta-llama/Meta-Llama-3-8B-Instruct,"[I, OM, MU, Ġes, Ġla, Ġab, rev, i, atura, Ġde,...","[27.5002536774, 26.0147819519, 33.4270744324, ...","IOMMU es la abreviatura de ""Input/Output Memor..."
4,ES,¿Quién ganó la Copa de Clubes de Asia 1997-98?,meta-llama/Meta-Llama-3-8B-Instruct,"[La, ĠCopa, Ġde, ĠClub, es, Ġde, ĠAsia, Ġ, 199...","[23.1833496094, 19.6576976776, 23.4080142975, ...",La Copa de Clubes de Asia 1997-98 fue ganada p...


In [4]:
# FUNCIONES

def noun_list(a, lang):
    """
    Filtra la pregunta y obtiene las PoST relevantes.
    
    a = list; Lista de preguntas del dataset
    lang = 'es' or 'en'; Idioma a trabajar
    """
    if lang == 'es':
        post_spacy = spacy.load("es_core_news_sm")
    else:
        post_spacy = spacy.load("en_core_web_sm")
    noun_list = []
    nums = ['0','1','2','3','4','5','6','7','8','9']

    for _ in a:
        doc = post_spacy(_)
        sub_noun = []
        for token in doc:
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "NUM":
                sub_noun.append(token.text)
            if token.pos_ == "ADJ" and token.text[0] in nums:
                sub_noun.append(token.text)
        noun_list.append(sub_noun)
    return noun_list

def keyword_por_preg(n_list):
    """
    Junta lista de PoST previo a pasarlo por el API de Wikipedia.
    
    n_list = list; Obtenida de la función noun_list().
    """
    keyword_list = []
    for i in n_list:
        keyword = ''
        for j in i:
            keyword = keyword + j + ' '
        keyword_list.append(keyword)
    return keyword_list

# Esta función regresa el título de la página de wikipedia más relevante.
# text = str ; El texto proveniente de keyword_list.
def get_wikipage(text, lang, page_total):
    """
    Regresa las n páginas de Wikipedia más relevantes al query

    text = str; Texto proveniente de la función keyword_por_preg()
    lang = 'es' or 'en'; Lenguaje necesario para wikipedia
    page_total = int; Cantidad de páginas a regresar
    """
    if lang == 'es':
        wikipedia.set_lang('es')
    if lang == 'en':
        wikipedia.set_lang('en')
    page_title = wikipedia.search(text, results = page_total)
    return page_title

In [5]:
# dataset = pandas column; Correspondiente a la columna de "model_input" del dataset en cuestión
# lang = str; Correspondiente al idioma enfocado

# Nos regresa la lista de resúmenes correspondientes a cada "model_input".

# Se puede optimizar verificando que el valor previo de la lista noun_list_perrona es el mismo que el actual.
# Perdón por poner hacer este algoritmo cuadrático. ¿Quién me va a castigar? ¿Dios? ¿Karla? ¿Diego? No creo, así que en realidad puedo hacer lo que yo quiera.

def wikipipeline(dataset, lang, page_count):
    """
    Genera los resúmenes que sirven como contexto de cada pregunta. 

    dataset = pd.DataFrame ; El nombre del dataset a procesar
    dataset = list; ya sea el nombre del dataset en formato dataset["model_input"] o list(set(dataset["model_input"]))
    lang = 'es' or 'en'; Idioma a trabajar, debe de coincidir con el del dataset para no generar algo incoherente
    page_count = int; Cantidad de páginas de Wikipedia a extraer
    """
    
    #input_column = dataset["model_input"]
    noun_list_perrona = noun_list(dataset, lang)
    key_list = keyword_por_preg(noun_list_perrona)

    resumen_list = []
    question_list = []
    iterador = 0
    for i in key_list:
        pages = get_wikipage(i, lang, page_count)
        resumen = ''
        for x in pages:
            try:
                page = wikipedia.WikipediaPage(x)
            except wikipedia.exceptions.DisambiguationError: # Se usa para evitar problemas al encontrar la página adecuada.
                print(i, page)
            page_sum = page.summary
            resumen = resumen + '' + page_sum
        resumen_list.append(resumen)
        question_list.append(dataset[iterador]) # Estos son las palabras claves de cada pregunta, no la pregunta misma.
        iterador += 1
    return resumen_list, question_list

In [6]:
%%time
cjto_en = list(set(df_en["model_input"]))
cjto_es = list(set(df_es["model_input"]))
sum_set_en, q_set_en = wikipipeline(cjto_en, 'en', 2)
sum_set_es, q_set_es = wikipipeline(cjto_es, 'es', 2)

C:\Users\FLopezP\Anaconda3\envs\ayuda_por_favor\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\FLopezP\Anaconda3\envs\ayuda_por_favor\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


name Edu Manga  <WikipediaPage 'List of active Swedish Navy ships'>
oriole  <WikipediaPage 'Eoin Morgan'>
estación  <WikipediaPage 'Twarde Pierniki Toruń'>
país ciudad Lugdunum  <WikipediaPage 'Apéndice (artrópodos)'>
CPU times: total: 5.61 s
Wall time: 1min 59s


In [10]:
%%time
summary_eng, q_en = wikipipeline(df_en["model_input"], 'en', 2)
summary_es, q_es = wikipipeline(df_es["model_input"], 'es', 2)

C:\Users\FLopezP\Anaconda3\envs\ayuda_por_favor\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\FLopezP\Anaconda3\envs\ayuda_por_favor\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


name Edu Manga  <WikipediaPage 'Municipalities of the canton of Fribourg'>
oriole  <WikipediaPage 'Index of Florida-related articles'>
oriole  <WikipediaPage 'Baltimore oriole'>
oriole  <WikipediaPage 'Baltimore oriole'>
oriole  <WikipediaPage 'Baltimore oriole'>
oriole  <WikipediaPage 'Baltimore oriole'>
name Edu Manga  <WikipediaPage 'Municipalities of the canton of Fribourg'>
name Edu Manga  <WikipediaPage 'Edu Manga'>
name Edu Manga  <WikipediaPage 'Edu Manga'>
name Edu Manga  <WikipediaPage 'Edu Manga'>
name Edu Manga  <WikipediaPage 'Edu Manga'>
name Edu Manga  <WikipediaPage 'Edu Manga'>
name Edu Manga  <WikipediaPage 'Edu Manga'>
name Edu Manga  <WikipediaPage 'Edu Manga'>
name Edu Manga  <WikipediaPage 'Edu Manga'>
name Edu Manga  <WikipediaPage 'Edu Manga'>
oriole  <WikipediaPage 'Index of Florida-related articles'>
oriole  <WikipediaPage 'Baltimore oriole'>
oriole  <WikipediaPage 'Baltimore oriole'>
oriole  <WikipediaPage 'Baltimore oriole'>
oriole  <WikipediaPage 'Baltimore

In [6]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16 = True)

def generate_embeddings(sum_list, q, ruta):
    """
    Genera dataframe de embeddings, y los guarda en un directorio de nuestra elección.

    sum_list = list ; Lista de resúmenes obtenida previamente.
    q = list ; Lista de preguntas obtenida previamente.
    ruta = str ; Directorio para guardar. 
    """
    len_list = [len(_) for _ in sum_list]
    max_length = max(len_list)

    embs = model.encode(
        sum_list,
        batch_size = 12,
        max_length = max_length,
    )['dense_vecs']

    embs_loco = [_ for _ in embs]
    dic = {'Embedding':embs_loco, 'Texto':sum_list, 'Keywords Pregunta': q}
    embs_df = pd.DataFrame(data=dic)
    embs_df.to_csv(ruta)
    print(f"Embedding guardados en la ruta {ruta} . Saludos")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [12]:
generate_embeddings(summary_eng, q_en, r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\eng_embs.csv')
generate_embeddings(summary_es, q_es, r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\es_embs.csv')

pre tokenize: 100%|██████████| 68/68 [00:00<00:00, 242.75it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 68/68 [00:16<00:00,  4.07it/s]


Embedding guardados en la ruta C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\eng_embs.csv . Saludos


Inference Embeddings: 100%|██████████| 41/41 [00:12<00:00,  3.24it/s]

Embedding guardados en la ruta C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\es_embs.csv . Saludos


In [14]:
generate_embeddings(sum_set_en, q_set_en, r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\eng_embs_singulares.csv')
generate_embeddings(sum_set_es, q_set_es, r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\es_embs_singulares.csv')

Embedding guardados en la ruta C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\eng_embs_singulares.csv . Saludos
Embedding guardados en la ruta C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\es_embs_singulares.csv . Saludos


In [ ]:
cjto_en = list(set(df_en["model_input"]))
cjto_es = list(set(df_es["model_input"]))
sum_set_en, q_set_en = wikipipeline(cjto_en, 'en', 2)
sum_set_es, q_set_es = wikipipeline(cjto_es, 'es', 2)
generate_embeddings(sum_set_en, q_set_en, r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\eng_embs_singulares.csv')
generate_embeddings(sum_set_es, q_set_es, r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\es_embs_singulares.csv')

In [8]:
def todojuntoalv(dataset, lang, num, ruta):
    """
    Ejecuta todo el pipeline (todo junto alv compa).
    
    dataset = pd.DataFrame[_column_name_] ;  Columna del dataset a trabajar
    lang = 'en' or 'es'; Lenguaje a trabajar.
    num = int; Cantidad de páginas de wikipedia a recolectar.
    ruta = str; Dirección de guardado de dataset. (DEBE DE TENER NOMBRE DEL ARCHIVO)
    """
    conjunto = list(set(dataset))
    sum_set, q_set = wikipipeline(conjunto, lang, num)
    generate_embeddings(sum_set, q_set, ruta)

In [7]:
val_ds = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\val_ds\mushroom.en-val.v2.jsonl', lines = True)
val_ds

,id,lang,model_input,model_output_text,model_id,soft_labels,hard_labels,model_output_logits,model_output_tokens
0,val-en-1,EN,What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,tiiuae/falcon-7b-instruct,"[{'start': 10, 'prob': 0.2, 'end': 12}, {'star...","[[25, 31], [45, 49], [69, 83]]","[-5.5669536591, -11.90533638, -13.0743436813, ...","[Pet, ra, Ġvan, ĠSto, ve, ren, Ġwon, Ġa, Ġsilv..."
1,val-en-2,EN,How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,tiiuae/falcon-7b-instruct,"[{'start': 4, 'prob': 0.2, 'end': 14}, {'start...","[[30, 31]]","[-6.199614048, -13.7564926147, -14.0058326721,...","[The, ĠE, lys, iph, ale, Ġorder, Ġcontains, Ġ,..."
2,val-en-3,EN,Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.6000000000000001, 'end...","[[0, 3], [9, 18], [63, 70], [78, 87]]","[-4.8190689087, -16.5279369354, -10.1344690323...","[Yes, ,, Ġall, Ġar, ach, n, ids, Ġhave, Ġanten..."
3,val-en-4,EN,When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,tiiuae/falcon-7b-instruct,"[{'start': 18, 'prob': 0.0909090909, 'end': 29...","[[29, 33]]","[-6.2555613518, -10.9392805099, -13.1335763931...","[Ch, ance, Ġthe, Ġrapper, Ġdebuted, Ġin, Ġ, 20..."
4,val-en-5,EN,What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.0909090909, 'end': 9},...","[[70, 227]]","[-3.9737114906, -12.9197320938, -7.2846975327,...","[The, ĠUN, ', s, ĠSustainable, ĠCity, Ġinitiat..."
5,val-en-6,EN,What are the four styles of Zhejiang cuisine?,Zhejing cuisine is known for its unique flavor...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.2727272727, 'end': 7},...","[[101, 109], [154, 158], [163, 170], [210, 220...","[-5.0601181984, -13.6542539597, -13.8530063629...","[Z, he, j, ing, Ġcuisine, Ġis, Ġknown, Ġfor, Ġ..."
6,val-en-7,EN,"If today is 14th October, and it is not a leap...","There are 365 days in a year. So, if today's d...",tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.1, 'end': 10}, {'start...","[[61, 65], [87, 90]]","[-3.6554422379, -15.4639282227, -8.6695766449,...","[There, Ġare, Ġ, 365, Ġdays, Ġin, Ġa, Ġyear, ...."
7,val-en-8,EN,Which network released the TV series of the Th...,The Puncher network that released this TV show...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.2727272727, 'end': 4},...","[[4, 11]]","[-2.758493185, -7.3909711838, -11.7919921875, ...","[The, ĠPun, cher, Ġnetwork, Ġthat, Ġreleased, ..."
8,val-en-9,EN,What is the population of the Spanish region o...,"As of 2021, the estimated population in the re...",tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.0909090909, 'end': 3},...","[[6, 10], [61, 72]]","[-5.1414809227, -14.3317422867, -8.5558795929,...","[As, Ġof, Ġ, 202, 1, ,, Ġthe, Ġestimated, Ġpop..."
9,val-en-10,EN,How many stages of labour are there in childbi...,There are typically four stages in labour: the...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.0909090909, 'end': 43}...","[[43, 116]]","[-4.2927789688, -13.868061065700001, -7.314704...","[There, Ġare, Ġtypically, Ġfour, Ġstages, Ġin,..."


In [9]:
%%time
todojuntoalv(val_ds["model_input"], 'en', 2, r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\val_embs\en_embs.csv')

C:\Users\FLopezP\Anaconda3\envs\ayuda_por_favor\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\FLopezP\Anaconda3\envs\ayuda_por_favor\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


name Edu Manga  <WikipediaPage 'Cerebral palsy'>
oriole  <WikipediaPage 'Edu Manga'>


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Embedding guardados en la ruta C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\val_embs\en_embs.csv . Saludos
CPU times: total: 8.55 s
Wall time: 1min 16s
